# **Chapter 8. [서버/통신] 사이트 접속자 수 맞추기 게임**


---
### 📝 **학습 목차**
> 8-1. 프로젝트 개요 <br>
> 8-2. 서버와 통신하기 - socket <br>
> 8-3. 여러명이 동시에 접속하려면? - select <br>
> 8-4. 플레이어 신호 수신하려면? - signal <br>
> **8-5. 프로젝트 실습**

## 8-5. 사이트 접속자 수 맞추기 게임 🎮

> ### 진행 순서
>  1. 게임 서버 만들기
>  2. 클라이언트 만들기

**[게임 규칙]**
- ① 서버에서 플레이어(클라이언트)의 접속을 기다린다.
- ② 플레이어가 서버에 접속하면 서버는 **접속자 수(정답)을 업데이트** 한다.
- ③ 접속한 플레이어는 예상되는 현재 사이트 접속자 수를 입력해서 서버로 보낸다.
- ④ 서버는 플레이어가 입력한 숫자가 정답보다 높을 때는 "너무 높아요"라고, 낮을 때는 "너무 낮아요"라고 응답한다.
- ⑤ 플레이어가 0을 입력하면 "종료"라고 응답하고 사이트 **접속자 수(정답)을 업데이트** 한다.
- ⑥ 클라이언트가 정답을 입력하면 "정답"이라고 응답하고 서버를 종료한다.

#### 8-5-1. 게임 서버 만들기

In [ ]:
# multi_play_game_server.py

import socket
import select
import random

HOST = 'localhost'   # 접속 서버 주소
PORT = 50007         # 클라이언트 접속을 대기하는 포트번호

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:    # 소켓 생성
    s.bind((HOST, PORT))   # 소켓과 포트 연결
    s.listen()             # 소켓 listening 시작
    print('서버가 시작되었습니다.')

    readsocks = [s]    # select 함수에서 관찰될 소켓 리스트 생성
    answers = {}       # 정답 딕셔너리 생성 (플레이어 별 정답 저장)
    num_clients = 0    # 현재 접속자 수

    while True:
        # select 함수로 소켓들의 상태를 확인
        readables, writeables, exceptions = select.select(readsocks, [], [])
        
        for sock in readables:
            if sock == s:  # 신규 클라이언트 접속
                newsock, addr = s.accept()
                num_clients += 1  # 접속자 수 증가
                print(f'클라이언트가 접속했습니다:{addr}, 정답은 {num_clients} 입니다.')
                readsocks.append(newsock)
                answers[newsock] = num_clients  # 신규 클라이언트 정답 생성
                for key in answers.keys():      # 기존 클라이언트 정답 업데이트
                    answers[key] = num_clients
                    
            else:  # 이미 접속한 클라이언트의 요청 (게임진행을 위한 요청)
                conn = sock
                data = conn.recv(1024).decode('utf-8')
                print(f'데이터:{data}')

                try:  # 숫자를 입력하지 않을 경우 응답 메세지
                    n = int(data)
                except ValueError:
                    conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
                    continue
                
                # 정답 가져오기
                answer = answers.get(conn)
                
                if n == 0:  # 클라이언트가 종료 0 를 입력할 경우
                    conn.sendall(f"종료".encode('utf-8'))
                    conn.close()
                    num_clients -= 1  # 접속자 수 감소
                    print(f'클라이언트가 게임을 종료했습니다:{addr}, 정답은 {num_clients} 입니다.')
                    
                    for key in answers.keys():  # 기존 클라이언트 정답 업데이트
                        answers[key] = num_clients
                    readsocks.remove(conn)  # 클라이언트 접속 해제시 readsocks에서 제거
                
                # 클라이언트의 입력값 채점
                elif n > answer:
                    conn.sendall("너무 높아요".encode('utf-8'))
                elif n < answer:
                    conn.sendall("너무 낮아요".encode('utf-8'))
                else:
                    conn.sendall("정답입니다 !".encode('utf-8'))


#### 8-5-2. 클라이언트 만들기

In [ ]:
# multi_play_game_client.py

import socket

HOST = 'localhost'   # 접속 서버 주소
PORT = 50007         # 클라이언트 접속을 대기하는 포트번호

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:    # 소켓 생성
    s.connect((HOST, PORT))  # 서버 연결

    while True:
        # 정답 입력
        n = input("예상 플레이어 숫자를 입력하세요(0은 게임포기):")
        
        # 입력값이 없을 경우
        if not n.strip():
            print("입력값이 잘못되었습니다.")
            continue
        
        # 서버 송신 (인코딩)
        s.sendall(n.encode('utf-8'))
        
        # 서버 응답 수신 (디코딩)
        data = s.recv(1024).decode('utf-8')
        
        # 서버 응답이 '정답'이거나 '종료' 이면 게임 종료
        print(f'서버응답:{data}')
        if data == "정답" or data == "종료":
            break